In [1]:
from langchain_core.prompts import ChatPromptTemplate

from enums import CollectionNames, EmbeddingModels
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_ollama import ChatOllama
from langchain_core.documents import Document

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_with_score_with_filters
)

import os
import re
import logging
from typing import List, Tuple

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte ich nicht gezielt reproduzieren)
logging.basicConfig(level=logging.INFO)

Nach den erfassten Metriken in der Evaluation des Retrievals hat sich herausgestellt, dass die Kombination aus dem deutschsprachigen Embedding-Modell, den Rekursiv gebildeten Chunks, den optimalen Filtern und den ursprünglichen Fragestellungen die besten Ergebnisse liefert.

Daher soll diese Kombination als Basis für das Generieren der Antworten genutzt werden.

Das genutzte Modell is das llama3.1:8b.

Das Modell wird über die Ollama-API angesprochen, hat 8 Milliarden Parameter und wird 4-Bit quantisiert genutzt.

Antworten werden in Textdateien gespeichert, um sie später in den Anhang der Arbeit einzufügen.


In [ ]:
# Konstanten
EMBEDDINGS = get_embeddings_function(model_name=EmbeddingModels.MIXEDBREAD.value)
QDRANT_URL = "http://localhost:6333"
COLLECTION = CollectionNames.MIXEDBREAD_RECURSIVE.value

FRAGEN = {
    1: "Wann kann ich mich einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit schreiben?",
    7: "Wie sind Portfolioprüfungen aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit?",
    10: "Wie lange dauern die Klausuren?"
}


# Alle relevanten Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )


FILTERS = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

LLAMA_MODELL = "llama3.1:8b"

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-15 11:33:56,925 [INFO] PyTorch version 2.6.0 available.
2025-03-15 11:33:57,177 [INFO] Use pytorch device_name: mps
2025-03-15 11:33:57,178 [INFO] Load pretrained SentenceTransformer: mixedbread-ai/deepset-mxbai-embed-de-large-v1


In [3]:
MAX_DOCUMENTS = 10

In [4]:
FH_PROMPT = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Du bist der Assistent der Fachhochschule Südwestfalen (FH-SWF).
Du beantwortest Fragen von Studieninteressierten und Studierenden.

Nutze den bereitgestellten Kontext, um die Frage konkret zu beantworten.
Wenn du die Frage nicht aus dem Kontext beantworten kannst, antworte mit "Das weiß ich leider nicht.", sonst nichts.
Erfinde niemals etwas, was nicht aus dem Kontext hervorgeht.

<|start_header_id|>user<|end_header_id|>
Kontext: {context}
Frage: {query}

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt_template = ChatPromptTemplate.from_template(FH_PROMPT)

In [ ]:
LLAMA_LLM = ChatOllama(
    model=LLAMA_MODELL,
    temperature=0,
    num_ctx=10000
)
# num_ctx legt das Kontextfenster in Tokens fest.
# Kann später reduziert werden, je nachdem wie viel Speicher benötigt wird. Kontext (max. 10 * 504) + Frage + Prompt sollte nie mehr als ca. 8k Tokens haben

In [ ]:
# Kontexte formatieren vor der Übergabe an das LLM
'''
Die drei Angaben aus den Metadaten füge ich hinzu. Ziel ist, dass Antworten spezifischer werden, wenn das Modell Studiengang, Standort un Abschluss zu jedem Chunk bekommt.
In den Antworten zeigte sich jedoch, dass dies nicht erfolgreich ist.
'''


def keep_relevant_metadata(documents: list[Document]) -> Tuple[List[Document], List[str]]:
    relevant_metadata = [
        "standort",
        "studiengang",
        "abschluss"
    ]
    links = []
    for doc in documents:
        if doc.metadata['link'] not in links:
            links.append(doc.metadata['link'])
        doc.metadata = {key: doc.metadata[key] for key in relevant_metadata}
    return documents, links


def format_page_content(documents: list[Document]) -> str:
    formatted_context = ""
    for doc in documents:
        formatted_context += f"Standort: {doc.metadata['standort']}, Studiengang: {doc.metadata['studiengang']}, Abschluss: {doc.metadata['abschluss']}\n{doc.page_content}\n\n"
    return formatted_context

In [8]:
def prepare_context(retrieved_documents: list[Document]) -> Tuple[str, List[str]]:
    documents, links = keep_relevant_metadata(retrieved_documents)
    prepared_context = format_page_content(documents)
    return prepared_context, links

In [9]:
# Qdrant-Clinet mit mixedbread-ai/deepset-mxbai-embed-de-large-v1, Recursive Chunks
best_client = get_qdrant_client(embeddings=EMBEDDINGS,
                                collection_name=COLLECTION,
                                qdrant_url=QDRANT_URL)

2025-03-15 11:34:25,302 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-15 11:34:25,308 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks "HTTP/1.1 200 OK"


In [10]:
# Ordner zum Speichern der Antworten
LLAMA_ANSWERS_DIR = "./antworten_llama_base_modell"

In [11]:
for frage in FRAGEN.items():
    frage_id, frage_text = frage
    print(f"Frage {frage_id}: {frage_text}")
    frage_filter = FILTERS[frage_id]
    retrieved_documents = retrieve_with_score_with_filters(client=best_client,
                                                           max_documents=MAX_DOCUMENTS,
                                                           filters=FILTERS[frage_id],
                                                           query=frage_text)
    documents_only = [doc for doc, _ in retrieved_documents]
    context, links = prepare_context(documents_only)
    prompt = rag_prompt_template.invoke({"context": context, "query": frage_text})
    response = LLAMA_LLM.invoke(prompt)
    answer = response.content
    input_tokens = response.usage_metadata["input_tokens"]
    print(f"Input Tokens: {input_tokens}")
    print(f"Antwort: {answer}\n{'-' * 50}")
    print("Für weitere Informationen besuchen Sie:")
    for link in links[
                :2]:  # Nur die ersten beiden Links anzeigen, da der MRR=0.911 ist, ist davon auszugehen, dass die relevanten Links in den ersten Positionen sind
        print(f"- {link}")

    clean_question_text = re.sub(r'[^\w\s-]', '', frage_text)
    filename = f"{frage_id}_{clean_question_text}.txt"
    filepath = os.path.join(LLAMA_ANSWERS_DIR, filename)

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"Frage {frage_id}: {frage_text}\n\n")
        file.write(f"Antwort:\n{answer}\n\n")

2025-03-15 11:34:33,751 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Frage 1: Wann kann ich mich einschreiben?


2025-03-15 11:35:01,605 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 5006
Antwort: Für Deutsche und Deutsche gleichgestellte Bewerber*innen:

- Wintersemester: i.d.R. Mitte Mai bis Ende August jeden Jahres
- Sommersemester: i.d.R. Anfang Dezember bis Ende Februar jeden Jahres

Für Bewerber*innen aus Nicht-EU-Ländern:

- Wintersemester: i.d.R. Anfang März bis 15. Juli jeden Jahres
- Sommersemester: i.d.R. Anfang November bis zum 15. Januar jeden Jahres
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php
- https://www.fh-swf.de/media/neu_np/hv_2/downloads_der_studiengangsseiten/fb_iw_1/elektrotechnik_m_eng_/FPO_MA_ETM_2017.pdf
Frage 2: Welche Literatur ist relevant für das Modul Mathematik?


2025-03-15 11:35:09,273 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-15 11:35:19,843 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Input Tokens: 5459
Antwort: Wenn Sie bei einer Prüfung krank sind, müssen Sie sich umgehend an einen Arzt wenden und ein ärztliches Attest beantragen. Dieses Attest sollte spätestens am Tag der Prüfung ausgestellt werden. Wenn Sie bereits vor der Prüfung erkrankt sind, können Sie sich bis 7 Tage vor der Prüfung online zurückziehen, ohne dass dies als Nachteil für Ihre Note gilt.

Wenn Sie jedoch nach Ablauf dieser Frist krank werden und die Prüfung nicht absagen konnten, müssen Sie ein ärztliches Attest vorlegen, das spätestens am Tag der Prüfung ausgestellt wurde. Dieses Attest muss unverzüglich eingereicht werden.

Es ist wichtig zu beachten, dass nur ein qualifiziertes Attest akzeptiert wird, in dem bescheinigt wird, dass Sie als prüfungsunfähig eingeschätzt wurden und warum dies der Fall ist.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/online_pruefungsorga/Ruecktritt-Info-2023.pd

2025-03-15 11:38:55,348 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-15 11:39:22,085 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 5432
Antwort: Die Inhalte des Moduls IT-Sicherheit umfassen:

- Grundbegriffe der IT-Sicherheit, Schutzziele, Angreifermodelle
- Prozessmodelle für Security Engineering, Misuse Cases, Attack Trees
- Sicherheitsmodelle (BLP & BIBA), IFC-Sprachen
- Asymmetrische und Symmetrische Verschlüsselung (RSA, AES)
- Hashfunktionen, Block- und Stromchiffren, Betriebsmodi
- kryptographische Protokolle (NSP, Kerberos, Diffie-Hellman)
- Zertifikate und digitale Signaturen
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/hagen/e___i/Modulhandbuch_Elektrotechnik_2024_Juli_2024.pdf
Frage 10: Wie lange dauern die Klausuren?


2025-03-15 11:39:32,990 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-15 11:39:50,133 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 3757
Antwort: Die Dauer der Klausur hängt von den Leistungspunkten des Moduls ab. 

- Bei Modulen mit 2 LP (Leistungspunkten) beträgt die Bearbeitungszeit mindestens 15 Minuten und höchstens 30 Minuten.
- Bei Modulen mit 5 LP beträgt die Bearbeitungszeit mindestens 60 Minuten und höchstens 120 Minuten.
- Bei Modulen mit 6 LP beträgt die Bearbeitungszeit mindestens 80 Minuten und höchstens 150 Minuten.
- Bei Modulen mit 7 LP beträgt die Bearbeitungszeit mindestens 100 Minuten und höchstens 180 Minuten.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/media/neu_np/hv_2/prfungsundnderungsordnungen/fbtbw_1/FPO_2018_WiIng_WiInf_IBE_IBI.pdf
- https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/hagen/tbw_neu/Modulhandbuch_Winf_FPO_2018_Stand_09.2024.pdf
